In [ ]:
import os
from pyspark.sql import SparkSession
import pandas as pd
os.environ["PYSPARK_PYTHON"] = "/usr/bin/python3.7"
# spark = SparkSession.builder.\
#         config('spark.executor.memory', '6g').\
#         config('spark.executor.cores', '8').\
#         config('spark.driver.memory','6g').\
#         config('spark.executor.instances', '30').\
#         appName('test').\
#         enableHiveSupport().getOrCreate()
spark = SparkSession.builder.\
        config('spark.executor.memory', '12g').\
        config('spark.executor.cores', '8').\
        config('spark.driver.memory','12g').\
        config('spark.executor.instances', '30').\
        appName('test').\
        enableHiveSupport().getOrCreate()

In [ ]:
train_data = spark.sql('select * from tmp.tmp_kmm_driver_8_27_train_2')
train_data = train_data.toPandas()
print("train_data.shape",train_data.shape)

In [ ]:
test_data = spark.sql('select * from tmp.tmp_kmm_driver_8_26_test_1')
test_data = test_data.toPandas()
print("train_data.shape",test_data.shape)


In [ ]:
train_data=train_data.fillna(0)
test_data=test_data.fillna(0)

In [ ]:
# val_data = spark.sql('select * from tmp.tmp_kmm_test3')
# val_data = val_data.toPandas()
# print("val_data.shape",val_data.shape)

val_data = spark.sql('select * from \
(select * ,row_number()over(ORDER BY user_guid) as guid from tmp.tmp_kmm_driver_test_826_4 where age between 20 and 50) t1 \
where t1.guid >60000000 and t1.guid <= 70000000  ')
# where t1.guid >30000000 and t1.guid <= 40000000  ')
# where t1.guid >40000000 and t1.guid <= 50000000  ')
# where t1.guid >50000000 and t1.guid <= 60000000  ')


val_data = val_data.toPandas()
print("val_data.shape",val_data.shape)
val_data=val_data.fillna(0)


In [ ]:
val_data1 = pd.DataFrame(val_data, columns=['age','sum_cal', 'sum_ride_time', 'sum_ride_number', 'sum_credit_score',
                                           'is_ride_card', 'consume_card_cnt', 'ev_consume_card_cnt', 'purchase_cnt',
                                           'ev_all_total_cnt', 'total_card_money', 'total_cnt', 'click'
                        ,'type1','type2','type3','type4','type5','type6','type7','type8','type9','type10','type11',
                                            'type12','type13','type14','type15','type16','type17','type18','type19',
#                                             'city1', 'city2', 'city3', 'city4','city5', 'city6', 'city7','city8', 'city9'
#                                             ,'city10', 'city11', 'city12','city13','city14','city15','city16','city17','city18'
#                                               ,'time1','time2','time3','time4'
                                               ])
val_data1['total_card_money']=val_data1['total_card_money'].astype('float')

print(val_data1.shape)
val_data1.head(2)


In [ ]:
train1_data = pd.DataFrame(train_data, columns=['age','sum_cal', 'sum_ride_time', 'sum_ride_number', 'sum_credit_score',
                                           'is_ride_card', 'consume_card_cnt', 'ev_consume_card_cnt', 'purchase_cnt',
                                           'ev_all_total_cnt', 'total_card_money', 'total_cnt', 'click'
                        ,'type1','type2','type3','type4','type5','type6','type7','type8','type9','type10','type11',
                                            'type12','type13','type14','type15','type16','type17','type18','type19',
#                                             'city1', 'city2', 'city3', 'city4','city5', 'city6', 'city7','city8', 'city9'
#                                             ,'city10', 'city11', 'city12','city13','city14','city15','city16','city17','city18'
#                                               ,'time1','time2','time3','time4'
                                               ])
# ,'type1','type2','type3','type4','type5','type6','type7','type8','type9','type10','type11','type12','type13',
#                        'type14','type15','type16','type17','type18','type19'
#  ,'city1', 'city2', 'city3', 'city4','city5', 'city6', 'city7','city8', 'city9'
#   ,'city10', 'city11', 'city12','city13','city14','city15','city16','city17','city18'
#                                  ,'time1','time2','time3','time4'
train1_data['total_card_money']=train1_data['total_card_money'].astype('float')
test1_data = pd.DataFrame(test_data, columns=['age', 'sum_cal', 'sum_ride_time', 'sum_ride_number', 'sum_credit_score',
                                           'is_ride_card', 'consume_card_cnt', 'ev_consume_card_cnt', 'purchase_cnt',
                                           'ev_all_total_cnt', 'total_card_money', 'total_cnt', 'click'
                ,'type1','type2','type3','type4','type5','type6','type7','type8','type9','type10','type11',
                                            'type12','type13','type14','type15','type16','type17','type18','type19',
#                                             'city1', 'city2', 'city3', 'city4','city5', 'city6', 'city7','city8', 'city9'
#                                             ,'city10', 'city11', 'city12','city13','city14','city15','city16','city17','city18'
#                                              ,'time1','time2','time3','time4'  
                                             ])
test1_data['total_card_money']=test1_data['total_card_money'].astype('float')

train_data_y = pd.DataFrame(train_data, columns=['lable'])
test_data_y = pd.DataFrame(test_data, columns=['lable'])
print(train1_data.shape)
print(test1_data.shape)
print(train_data_y.shape)
print(test_data_y.shape)
train1_data.head(2)

In [ ]:
from sklearn.metrics import confusion_matrix
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score, precision_score, accuracy_score, recall_score, f1_score, roc_auc_score
from xgboost import XGBClassifier

print("---------begin---------")
# model = LogisticRegression()
model=XGBClassifier(n_jobs=8,seed=2019,max_depth=3)
model.fit(train1_data, train_data_y)
# lr_model.fit(train2, train_data_y)

print("---------train over---------")
# print("w: ", model.coef_)
# print("b: ", model.intercept_)
precision = model.predict(test1_data) 
pro = model.predict_proba(test1_data)
target = np.array(test_data_y)

print('精确率:{}'.format(precision_score(target,precision )))
print('准确率:{}'.format(accuracy_score(target,precision )))
print('召回率:{}'.format(recall_score(target,precision )))
print('f1:{}'.format(f1_score(target, precision)))
print('auc:{}'.format(roc_auc_score(target, pro[:, 1])))

In [ ]:
result = pd.concat(
[pd.DataFrame(test_data, columns=['user_guid']),pd.DataFrame(pro, columns=['预测为0的概率', '预测为1的概率'])], axis=1)
result1 = result[(result['预测为1的概率'] >= 0.9)]
result1.shape
# result1.to_csv(r'result.csv', index=True, header=True )
#  pd.to_csv(outputfile, mode='a', index=False, header=False)　

In [ ]:
from sklearn import metrics
def sta(res_0, a, b):
    q1 = res_0[(res_0['预测为1的概率'] >= a) & (res_0['预测为1的概率'] < b)]
    mat_0 = confusion_matrix(q1['label'], q1['预测结果'], labels=[1, 0])

    new = pd.DataFrame(mat_0.reshape(1, 4), columns=['预测转化_实际转化', '预测未转化_实际转化', '预测转化_实际未转化', '预测未转化_实际未转化'],
                       index=['预测概率{0}-{1}'.format(a, b)])
    new['准确率'] = metrics.accuracy_score(q1['label'], q1['预测结果'])
    metrics.accuracy_score(q1['label'], q1['预测结果'])
    return new

res_3 = pd.concat(
        [pd.DataFrame(target, columns=['label']), pd.DataFrame(precision, columns=['预测结果']), pd.DataFrame(pro, columns=['预测为0的概率', '预测为1的概率'])], axis=1)

df3 = sta(res_3, 0.96, 1)
for a, b in [[0.94, 0.96],[0.90, 0.94],[0.85, 0.9],[0.8, 0.85], [0.7, 0.8], [0.6, 0.7], [0.5, 0.6], [0.4, 0.5], [0.3, 0.4], [0.2, 0.3], [0.1, 0.2], [0, 0.1]]:
    df_new = pd.concat([df3, sta(res_3, a, b)])
    df3 = df_new
    

In [ ]:
df3

In [ ]:
from sklearn.metrics import confusion_matrix
mat = confusion_matrix(target ,precision, labels=[1, 0])
pd.DataFrame(mat, columns=['预测转化', '预测未转化'], index=['实际转化', '实际未转化'])

In [ ]:
val_pro = model.predict_proba(val_data1)

In [ ]:
result = pd.concat(
[pd.DataFrame(val_data, columns=['user_guid']),pd.DataFrame(val_pro, columns=['预测为0的概率', '预测为1的概率'])], axis=1)
result1 = result[(result['预测为1的概率'] >= 0.96)]
result1.shape

In [ ]:
result1.to_csv(r'result1.csv',mode='a', index=True, header=True )  #,mode='a'

In [ ]:
result2 = result[(result['预测为1的概率'] >= 0.94)& (result['预测为1的概率'] <0.96)]
result2.shape

In [ ]:
result2.to_csv(r'result2.csv', mode='a',index=True, header=True )

In [ ]:
result1.to_csv(r'result.csv', index=True, header=True )
#  pd.to_csv(outputfile, mode='a', index=False, header=False)　

In [ ]:
result = pd.concat(
[pd.DataFrame(val_data, columns=['user_guid','age']),pd.DataFrame(val_pro, columns=['预测为0的概率', '预测为1的概率'])], axis=1)
result1 = result[(result['预测为1的概率'] >= 0.9)&(result['age'] >= 20)&(result['age'] <=50) ]
result1.shape